<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/model_diagnostics/model_diagnostics_demo.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/master/examples/model_diagnostics/model_diagnostics_demo.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>



# Model Diagnostics


Throughout the process of training your machine learning (ML) model, you may want to investigate your model's failures in order to understand which areas need improvement. Looking at an error analysis after each training iteration can help you understand whether you need to revise your annotations, make your ontology more clear, or create more training data that targets a specific area.
Labelbox now offers a Model Diagnostics tool that analyzes the performance of your model's predictions in a single interface.
With Model Diagnostics, you can:
*   Inspect model behavior across experiments
*   Adjust model hyperparameters and visualize model failures
*   Use the Python SDK to create the analysis pipeline

## How it works

Configuring Model Diagnostics is all done via the SDK. We have created a Google colab notebook to demonstrate this process. The notebook also includes a section that leverages MAL in order to quickly create ground truth annotations.
An Experiment is a specific instance of a model generating output in the form of predictions.
In Labelbox, the `Model` object represents your ML model and it is what you'll be performing experiments on. It references a set of annotations specified by an ontology. 
The `Model Run` object represents the experiment itself. It is a specific instance of a `Model` with preconfigured hyperparameters (training data). You can upload inferences across each `Model Run`, filter by IoU score, and compare your model's predictions against the annotations from your training data.

## Steps
1. Make sure you are signed up for the beta. If not navigate here https://labelbox.com/product/model-diagnostics
2. Have a set of ground truth labels in a project
3. Install the latest SDK release
4. Create a `Model`
5. Create a `Model Run`
6. Compute predictions
7. Compute model performance metrics
8. Upload labels, predictions, and metrics
9. Navigate to the `Models` tab on Labelbox

## Best practices
Currently there is a limit of 2000 images per model run. We suggest uploading lower performing examples from your test set.


## Environment Setup

Install dependencies

In [ ]:
!pip install -q "labelbox[data]" \
             scikit-image \
             tensorflow

In [ ]:
# Run these if running in a colab notebook
COLAB = "google.colab" in str(get_ipython())

if COLAB:
    !git clone https://github.com/Labelbox/labelbox-python.git
    !cd labelbox-python
    !mv labelbox-python/examples/model_assisted_labeling/*.py .

Import libraries

In [ ]:
import sys

sys.path.append('../model_assisted_labeling')

import uuid
import numpy as np
from skimage import measure
from tqdm import notebook
import requests

import labelbox as lb
import labelbox.types as lb_types
from labelbox.data.metrics.group import get_label_pairs
from labelbox.data.metrics import feature_miou_metric, feature_confusion_matrix_metric

try:
    from image_model import predict, load_model
except ModuleNotFoundError:
    # !git clone https://github.com/Labelbox/labelbox-python.git
    # !cd labelbox-python && git checkout mea-dev
    # !mv labelbox-python/examples/model_assisted_labeling/*.py .
    raise Exception("You will need to run from the labelbox-python git repo")

Configure client

In [ ]:
PROJECT_NAME = "Diagnostics Demo"
MODEL_NAME = "MSCOCO-Mapillary"
MODEL_VERSION = "0.0.0"

API Key

Provide a valid api key below in order to properly connect to the Labelbox Client.

In [ ]:
# Add your api key
API_KEY = None
client = lb.Client(api_key=API_KEY)
load_model()  # initialize Tensorflow Model

In [ ]:
# Configure for whatever combination of tools and class names that you would like.
class_mappings = {
    1: {
        "name": 'person',
        "kind": Tool.Type.POLYGON
    },
    2: {
        "name": 'bicycle',
        "kind": Tool.Type.SEGMENTATION,
        'color': 64
    },
    3: {
        "name": 'car',
        "kind": Tool.Type.BBOX
    },
    4: {
        "name": 'motorcycle',
        "kind": Tool.Type.BBOX
    },
    6: {
        "name": 'bus',
        "kind": Tool.Type.POLYGON
    },
    7: {
        "name": 'train',
        "kind": Tool.Type.POLYGON
    },
    8: {
        "name": 'truck',
        "kind": Tool.Type.POLYGON
    },
    10: {
        "name": 'traffic light',
        "kind": Tool.Type.POINT
    },
    11: {
        "name": 'fire hydrant',
        "kind": Tool.Type.BBOX
    },
    13: {
        "name": 'stop sign',
        "kind": Tool.Type.SEGMENTATION,
        'color': 255
    },
    14: {
        "name": 'parking meter',
        "kind": Tool.Type.POINT
    },
    28: {
        "name": 'umbrella',
        "kind": Tool.Type.SEGMENTATION,
        'color': 128
    },
    31: {
        "name": 'handbag',
        "kind": Tool.Type.POINT
    },
}

## Create Predictions
* Loop over data_rows, make predictions, and create ndjson

In [ ]:
# --- setup dataset ---
# load mapillary sample
sample_csv_url = "https://raw.githubusercontent.com/Labelbox/labelbox-python/master/examples/assets/mapillary_sample.csv"
with requests.get(sample_csv_url, stream=True) as r:
    image_data = [
        row.split(',')
        for row in (line.decode('utf-8') for line in r.iter_lines())
    ]

In [ ]:
predictions = list()
for (image_url, external_id) in notebook.tqdm(image_data[:10]):
    image = lb_types.ImageData(url=image_url, external_id=external_id)
    height, width = image.value.shape[:2]
    prediction = predict(np.array([image.im_bytes]),
                         min_score=0.5,
                         height=height,
                         width=width)
    boxes, classes, seg_masks = prediction["boxes"], prediction[
        "class_indices"], prediction["seg_masks"]
    annotations = []
    for box, class_idx, seg in zip(boxes, classes, seg_masks):
        if class_idx in class_mappings:
            class_info = class_mappings.get(class_idx)
            if class_info['kind'] == lb.Tool.Type.POLYGON:
                contours = measure.find_contours(seg, 0.5)
                pts = contours[0].astype(np.int32)
                value = lb_types.Polygon(points=[
                    lb_types.Point(x=x, y=y) for x, y in np.roll(pts, 1, axis=-1)
                ])
            elif class_info['kind'] == lb.Tool.Type.BBOX:
                value = lb_types.Rectangle(start=lb_types.Point(x=box[1], y=box[0]),
                                  end=lb_types.Point(x=box[3], y=box[2]))
            elif class_info['kind'] == lb.Tool.Type.POINT:
                value = lb_types.Point(x=(box[1] + box[3]) / 2.,
                              y=(box[0] + box[2]) / 2.)
            elif class_info['kind'] == lb.Tool.Type.SEGMENTATION:
                value = lb_types.Mask(mask=lb_types.MaskData.from_2D_arr(seg *
                                                       class_info['color']),
                             color=(class_info['color'],) * 3)
            else:
                raise ValueError(
                    f"Unsupported kind found. {class_info['kind']}")
            annotations.append(
                lb_types.ObjectAnnotation(name=class_info['name'], value=value))
    predictions.append(lb_types.Label(data=image, annotations=annotations))

## Setup a project

In [ ]:
# --- Use the class mapping specified above ( Will include all specified classes )
tools = []
for target in class_mappings.values():
    tools.append(lb.Tool(tool=target['kind'], name=target["name"]))
ontology_builder = lb.OntologyBuilder(tools=tools)

# --- Optionally Setup ontology from predictions ( Only will include predicted classes )
#ontology_builder = predictions.get_ontology()

In [ ]:
print(f"Setting up: {PROJECT_NAME}")

project = client.create_project(name=PROJECT_NAME, media_type=lb.MediaType.Image)
editor = next(
    client.get_labeling_frontends(where=lb.LabelingFrontend.name == "Editor"))
project.setup(editor, ontology_builder.asdict())

dataset = client.create_dataset(name="Mapillary Diagnostics Demo")
print(f"Dataset Created: {dataset.uid}")
project.create_batches_from_dataset("batch", dataset.uid)

## Prepare for upload
* Our local annotations need the following:
    1. signed url for segmentation masks
    2. data rows in labelbox
    3. feature schema ids

In [ ]:
signer = lambda _bytes: client.upload_data(content=_bytes, sign=True)
predictions.add_url_to_masks(signer) \
         .add_url_to_data(signer) \
         .add_to_dataset(dataset, client.upload_data)

## **Optional** - Create labels with [Model Assisted Labeling](https://docs.labelbox.com/en/core-concepts/model-assisted-labeling)

* Pre-label image so that we can quickly create ground truth
* Create ground truth data for Model Diagnostics
* Click on link below to label

In [ ]:
RUN_MAL = True
if RUN_MAL:
    project.enable_model_assisted_labeling()
    # Convert from annotation types to import format
    upload_task = lb.MALPredictionImport.create_from_objects(
        client, project.uid, f'mal-import-{uuid.uuid4()}', predictions)
    upload_task.wait_until_done()
    print(upload_task.state, '\n')

In [ ]:
print(f"https://app.labelbox.com/go-label/{project.uid}")

## Export Labels

We do not support `Skipped` labels and have a limit of **2000**

In [ ]:
MAX_LABELS = 2000
labels = [
    l for idx, l in enumerate(project.label_generator()) if idx < MAX_LABELS
]

## Setup Model & Model Run

In [ ]:
lb_model = client.create_model(name=MODEL_NAME,
                               ontology_id=project.ontology().uid)
lb_model_run = lb_model.create_model_run(MODEL_VERSION)

Select label ids to upload

In [ ]:
lb_model_run.upsert_labels([label.uid for label in labels])

### Compute Metrics

In [ ]:
pairs = get_label_pairs(labels, predictions, filter_mismatch=True)
for (ground_truth, prediction) in pairs.values():
    metrics = []
    metrics.extend(
        feature_miou_metric(ground_truth.annotations, prediction.annotations))
    metrics.extend(
        feature_confusion_matrix_metric(ground_truth.annotations,
                                        prediction.annotations))
    prediction.annotations.extend(metrics)

### Upload to Labelbox

In [ ]:
upload_task = lb_model_run.add_predictions(
    f'diagnostics-import-{uuid.uuid4()}', predictions)
upload_task.wait_until_done()
print(upload_task.state)

### Open Model Run

In [ ]:
for idx, model_run_data_row in enumerate(lb_model_run.model_run_data_rows()):
    if idx == 5:
        break
    print(model_run_data_row.url)

### Export model run labels

In [ ]:
MODEL_ID = ''
MODEL_RUN_ID = ''
model = client.get_model(MODEL_ID)

model_run = next(filter(lambda run: run['id'] == MODEL_RUN_ID, model.model_runs), None)
labels = model_run.export_labels(download=True)